# Домашнее задание к лекции "Статистика. Практика"

## Задание 1

Вернемся к [набору данных о видеоиграх](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/vgsales.csv).

Ответьте на следующие вопросы:

1) Как критики относятся к спортивным играм?  
2) Критикам нравятся больше игры на PC или на PS4?  
3) Критикам больше нравятся стрелялки или стратегии?  

Для каждого вопроса:
- сформулируйте нулевую и альтернативную гипотезы;
- выберите пороговый уровень статистической значимости;
- опишите полученные результаты статистического теста.

In [123]:
import pandas as pd
import numpy as np
from scipy import stats as st
games = pd.read_csv('vgsales.csv', encoding='utf-8')
games_df = games.filter(items = ['Platform', 'Genre', 'Critic_Score'])

games_df = games_df.dropna()

games_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8137 entries, 0 to 16709
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Platform      8137 non-null   object 
 1   Genre         8137 non-null   object 
 2   Critic_Score  8137 non-null   float64
dtypes: float64(1), object(2)
memory usage: 254.3+ KB


### Как критики относятся к спортивным играм? 

In [124]:
games_df

,Platform,Genre,Critic_Score
0,Wii,Sports,76.0
2,Wii,Racing,82.0
3,Wii,Sports,80.0
6,DS,Platform,89.0
7,Wii,Misc,58.0
...,...,...,...
16700,PC,Shooter,61.0
16701,GC,Puzzle,53.0
16702,DS,Puzzle,48.0
16706,PC,Strategy,60.0


### 2. Критикам нравятся больше игры на PC или на PS4?

H0: Критикам нравятся игры на PC и на PS4 одинаково  
H1: Критики имеют разлитчное отношение к играм на PC и играм на PS4  
За alpha будем принимать 0.05

In [107]:
#сформируем две выборки с оценками критиков для каждой из платформ и удалим пустые строки
games_df_pc = games_df[games_df.Platform == 'PC'].dropna()
games_df_ps4 = games_df[games_df.Platform == 'PS4'].dropna()

In [108]:
alpha = 0.05
result = st.ttest_ind(games_df_pc['Critic_Score'], games_df_ps4['Critic_Score'])
print(result)

if (result.pvalue < alpha):
    if (result.statistic > 0):
        print('Отвергаем нулевую гипотезу, критикам больше нравятся игры на РС')
    elif (result.statistic < 0):
        print('Отвергаем нулевую гипотезу, критикам больше нравятся игры на PS4')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=4.52230721935523, pvalue=6.8897336450541214e-06)
Отвергаем нулевую гипотезу, критикам больше нравятся игры на РС


### 3. Критикам больше нравятся стрелялки или стратегии? 

H0: Критикам одинаково нравятся стрелялки и стратегии  
H1: Критики имеют разлитчное отношение к стрелялкам и стратегиям  
За alpha будем принимать 0.05

In [117]:
#сформируем две выборки с оценками критиков для каждой категории и удалим пустые строки
games_df_shooter = games_df[games_df.Genre == 'Shooter'].dropna()
games_df_strategy = games_df[games_df.Genre == 'Strategy'].dropna()
print(games_df_shooter.Critic_Score.mean())
print(games_df_strategy.Critic_Score.mean())

70.90293453724605
72.76408450704226


In [115]:
alpha = 0.05
result = st.ttest_ind(games_df_shooter['Critic_Score'], games_df_strategy['Critic_Score'])
print(result)

if (result.pvalue < alpha):
    if (result.statistic > 0):
        print('Отвергаем нулевую гипотезу, критикам больше нравятся стрелялки')
    elif (result.statistic < 0):
        print('Отвергаем нулевую гипотезу, критикам больше нравятся стратегии')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=-1.9760868702428087, pvalue=0.04837995015879659)
Отвергаем нулевую гипотезу, критикам больше нравятся стратегии


In [112]:
print(games_df.Genre.unique())

['Sports' 'Racing' 'Platform' 'Misc' 'Action' 'Puzzle' 'Shooter'
 'Fighting' 'Simulation' 'Role-Playing' 'Adventure' 'Strategy']


## Задание 2

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные [здесь](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/spam.csv)) по признаку спама. Для этого:

1) Привидите весь текст к нижнему регистру;  
2) Удалите мусорные символы;  
3) Удалите стоп-слова;  
4) Привидите все слова к нормальной форме;  
5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:  

```
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
```

Можете поэкспериментировать с параметрами [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html);  

6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите `random_state=42`. Используйте [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html);  

7) Постройте модель [логистической регрессии](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), укажите `random_state=42`, оцените ее точность на тестовых данных;  

8) Опишите результаты при помощи [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html?highlight=confusion_matrix#sklearn.metrics.confusion_matrix);  

9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [78]:
import pandas as pd
import numpy as np
spam = pd.read_csv('spam.csv', encoding='utf-8')
# приведем все слова к нижнему регистру
spam['Message'] = spam['Message'].str.lower()

spam

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,this is the 2nd time we have tried 2 contact u...
5568,ham,will ü b going to esplanade fr home?
5569,ham,"pity, * was in mood for that. so...any other s..."
5570,ham,the guy did some bitching but i acted like i'd...


In [79]:
print(spam.Category.unique())

['ham' 'spam']


In [ ]:
# уберем все служебные символы
print(len(texts_lower_case[15]))
only_words_text = [re.sub('[\W_]+',' ', text) for text in texts_lower_case]
# проверим
print(len(only_words_text[15]))
only_words_text[15]